#  
<!-- https://medium.com/the-prefect-blog/the-simple-guide-to-productionizing-data-workflows-with-docker-31a5aae67c0a -->

<p align="center">
    <img src="https://miro.medium.com/max/504/1*iBGlEPUruUqqT5NreeEF8g.png" width=200>
</p>

## **Exercise**: dockerizing some python

Overview:
- [what is docker?](#what-is-docker)
- [Why use docker?](#why-use-docker)
- [How to build a docker image](#how-to-build-a-docker-image)
- [Pushing an image to a registry](#how-to-push-an-image-to-a-registry)
- [**Exercise**: Build a simple custom image of your own](#exercise-build-your-own-custom-image)

## what is docker?
it is an open-source containerization technology especially useful for cloud-based ecosystems

## why use docker?
- Simplify cloud-native development
- Portability
- Reproducability
- Infinitely custom

## how can we **build** a docker image?
we tell docker what to build!

`Dockerfile`
```Dockerfile
# starting
FROM base_image:andmyTag

COPY myModule2 /root/or/somewhere_on_my_new_container

# and for myModule2's dependencies, we need to
RUN pip install -r requirements.txt

# and then maybe some commands to run when the container starts, like a
CMD echo 'ing some handy dandy message like "cats!" or "im cool!"'
```
## **push** an image to a registry

### Auth
<!-- thoughts on actually having people login/signup w dockerhub? -->
We need to auth our CLI session with our image repository otherwisesuperconvenientfreestorage

```console
docker login
```
sign up etc

### Build and Tag
```console
docker build . --tag zzstoatzz/prefect-imgs:dev
```

## other cool images (prolly ML stuff) + what they do